In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os 
import cv2
from random import shuffle
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, Input
from tensorflow.keras import activations
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [ ]:
GOOD_POSE_DIR = 'good_pose'
BAD_POSE_DIR = 'bad_pose'
IMG_SIZE = 256

In [ ]:
def label_image(image):
    """
    Function to encode and give a label to an image.
    image : (String) -> represents the image name.
    return : (Int) -> 0/1 encoding for a bad/good pose image.
    """
    world_label = image.split('_')[0]
    if world_label == 'good':
        return 1
    return 0

In [ ]:
def process_data(form):
    """
    Function to process the data and save it in a numpy array form. (friendlier to neural networks)
    form : (String) -> represents the type of images it receives
    """
    data = []
    if form == 'good':
        dir = os.listdir(GOOD_POSE_DIR)
    else:
        dir = os.listdir(BAD_POSE_DIR)

    for image in dir:
        label = label_image(image)
        if form == 'good':
            path = os.path.join(GOOD_POSE_DIR, image)
        else:
            path = os.path.join(BAD_POSE_DIR, image)
        img = cv2.resize(cv2.imread(path, cv2.IMREAD_COLOR), (IMG_SIZE, IMG_SIZE))
        data.append([np.array(img), label])
    
    if form == 'good':
        np.save('good_pose_data.npy', data)
    else:
        np.save('bad_pose_data.npy', data)

In [ ]:
# Loading the data
good_pose = np.load('good_pose_data.npy', allow_pickle=True)
bad_pose = np.load('bad_pose_data.npy', allow_pickle=True)

In [ ]:
# Mixing the good pose data and bad pose data
good_and_bad_pose = np.concatenate((good_pose, bad_pose), axis=0)

# Split the data in train/test.
data = good_and_bad_pose[:, 0]
labels = good_and_bad_pose[:, 1]

X_train, X_valid, y_train, y_valid = train_test_split(data, labels, train_size=0.8)

y_train = np.asarray(y_train).astype(np.float32)
y_valid = np.asarray(y_valid).astype(np.float32)

## Running Openpose over the images to get the data

In [ ]:
thr = 0.2
BODY_PARTS = { "Nose": 0, "Neck": 1, "RShoulder": 2, "RElbow": 3, "RWrist": 4,
               "LShoulder": 5, "LElbow": 6, "LWrist": 7, "RHip": 8, "RKnee": 9,
               "RAnkle": 10, "LHip": 11, "LKnee": 12, "LAnkle": 13, "REye": 14,
               "LEye": 15, "REar": 16, "LEar": 17, "Background": 18 }

POSE_PAIRS = [ ["Neck", "RShoulder"], ["Neck", "LShoulder"], ["RShoulder", "RElbow"],
               ["RElbow", "RWrist"], ["LShoulder", "LElbow"], ["LElbow", "LWrist"],
               ["Neck", "RHip"], ["RHip", "RKnee"], ["RKnee", "RAnkle"], ["Neck", "LHip"],
               ["LHip", "LKnee"], ["LKnee", "LAnkle"], ["Neck", "Nose"], ["Nose", "REye"],
               ["REye", "REar"], ["Nose", "LEye"], ["LEye", "LEar"] ]

In [ ]:
# Load the weigths
net = cv2.dnn.readNetFromTensorflow('graph_opt.pb')

In [ ]:
def get_openpose_data(frame):
    """
    Function to run the openpose model over an image and get the coordonates for all 19 keypoints
    frame: (List) -> The image
    """
    frameWidth = frame.shape[1]
    frameHeight = frame.shape[0]
    
    net.setInput(cv2.dnn.blobFromImage(frame, 1.0, (IMG_SIZE, IMG_SIZE), (127.5, 127.5, 127.5), swapRB=True, crop=False))
    out = net.forward()
    out = out[:, :19, :, :]

    assert(len(BODY_PARTS) == out.shape[1])

    points = []
    for i in range(len(BODY_PARTS)):
        heatMap = out[0, i, :, :]

        _, conf, _, point = cv2.minMaxLoc(heatMap)
        x = (frameWidth * point[0]) / out.shape[3]
        y = (frameHeight * point[1]) / out.shape[2]

        points.append([int(x), int(y)] if conf > thr else None)

    return points

In [226]:
def multiple_openpose_data(data):
    x_data = []
    scaler = MinMaxScaler()

    for idx1, image in enumerate(X_train):
        openpose_data = get_openpose_data(image)
        for idx2, coords in enumerate(openpose_data):
            if coords == None:
                openpose_data[idx2] = [0, 0]
        if idx1 == 0:
            scaler.fit(openpose_data)
        openpose_data = scaler.transform(openpose_data)[:-1]
        X_train_openpose.append(openpose_data)
    
    x_data = np.array(x_data).astype(float)
    return x_data

In [ ]:
scaler = MinMaxScaler()
X_train_openpose = []
X_valid_openpose = []

# Get the keypoints and for the unavailable keypoints transform NoneType to 0
# Plus we scale them [0, 1]
for idx1, image in enumerate(X_train):
    openpose_data = get_openpose_data(image)
    for idx2, coords in enumerate(openpose_data):
        if coords == None:
            openpose_data[idx2] = [0, 0]
    if idx1 == 0:
        scaler.fit(openpose_data)
    openpose_data = scaler.transform(openpose_data)[:-1]
    X_train_openpose.append(openpose_data)

for image in X_valid:
    openpose_data = get_openpose_data(image)
    for idx, coords in enumerate(openpose_data):
        if coords == None:
            openpose_data[idx] = [0, 0]
    openpose_data = scaler.transform(openpose_data)[:-1]
    X_valid_openpose.append(openpose_data)
 

X_train_openpose = np.array(X_train_openpose).astype(float)
X_valid_openpose = np.array(X_valid_openpose).astype(float)

In [ ]:
# Visualizing the keypoints
for coords in X_train_openpose:
    for coord, body_part in zip(coords, BODY_PARTS.keys()):
        print(f'{body_part} -> {coord}')
    print('\n')

## The fully conected model that takes the keypoint coordinates

In [ ]:
model = Sequential()
model.add(Input(shape=(18,2)))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='relu'))
optimizer = tf.keras.optimizers.Adam(lr=0.001)
model.compile(loss='binary_crossentropy',
                optimizer=optimizer,
                metrics=['accuracy'])

In [ ]:
model.fit(X_train_openpose, y_train, epochs=200)

In [ ]:
output = model.predict(X_valid_openpose)

In [ ]:
print(confusion_matrix(y_valid, output.round()))
print(accuracy_score(y_valid, output.round()))

In [ ]:
# Saving the model
model.save('openpose_bicepscurl_nobg')

## Working on the video preprocessing

In [ ]:
def fragment_video(video_name, save_location, interval):
    """
    video_name: String -> path to the video
    save_location: Strin -> path to the save location
    interval: Int -> interval in ms
    """
    vidcap = cv2.VideoCapture(video_name)
    success, image = vidcap.read()

    # Change the current directory.
    #os.chdir(save_location)

    # While we have frames, we read and save them.
    count = 1
    while success:
        vidcap.set(cv2.CAP_PROP_POS_MSEC, (count*interval))
        cv2.imwrite(f'frame{count}.jpg', image)
        success, image = vidcap.read()
        count += 1

In [ ]:
def read_frames(location):
    frames = []
    dir = os.listdir(location)

    for frame in dir:
        path = os.path.join(location, frame)
        frame = cv2.resize(cv2.imread(path, cv2.IMREAD_COLOR), (IMG_SIZE, IMG_SIZE))
        frames.append(frame)
    return frames

In [221]:
fragment_video('D:\College\coding (Python)\PoseNet - Biceps Curl\\videos\\2.mp4', 'frames', 250)

In [222]:
input_data = read_frames('D:\College\coding (Python)\PoseNet - Biceps Curl\\frames')

In [223]:
x_valid_video = []

for image in input_data:
    openpose_data = get_openpose_data(image)
    for idx, coords in enumerate(openpose_data):
        if coords == None:
            openpose_data[idx] = [0, 0]
    openpose_data = scaler.transform(openpose_data)[:-1]
    x_valid_video.append(openpose_data)

x_valid_video = np.array(x_valid_video).astype(float)

In [224]:
output = model.predict(x_valid_video)

In [225]:
print(output)

[[0.73810977]
 [2.9458427 ]
 [2.9144278 ]
 [0.        ]
 [1.0992996 ]
 [1.7052757 ]
 [0.17849872]
 [2.9229093 ]
 [2.217883  ]
 [0.36903706]
 [0.40409335]
 [0.        ]
 [1.0808004 ]
 [0.        ]
 [1.2917217 ]
 [2.8783221 ]
 [0.        ]
 [0.39451832]
 [1.7501711 ]
 [2.046883  ]
 [2.2756276 ]
 [2.5198426 ]
 [1.5046622 ]]
